In [ ]:
Smile detector
Image Classification -> Supervised->Classification -> Binary Classification
Folosim retea preantrenata si eliminam ultimele layere liniare, Data generator/ Data loader

In [1]:
import pandas as pd
from PIL import Image
import torch
import torchvision
from torchvision.io import read_image
from torchvision.models import regnet_y_400mf 
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
import os

In [16]:
if torch.cuda.is_available(): 
  dev = "cuda:0" 
else: 
  dev = "cpu" 
device = torch.device(dev)

In [17]:
dev

'cuda:0'

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
df = pd.read_csv("/content/drive/MyDrive/DataScience/datasets/Celeba/list_attr_celeba.csv")

In [3]:
model = regnet_y_400mf(weights='IMAGENET1K_V1', )

Downloading: "https://download.pytorch.org/models/regnet_y_400mf-c65dace8.pth" to /root/.cache/torch/hub/checkpoints/regnet_y_400mf-c65dace8.pth


  0%|          | 0.00/16.8M [00:00<?, ?B/s]

In [ ]:
print(model)

In [5]:
model.fc = torch.nn.Linear(in_features=440, out_features=2, bias=True)

In [ ]:
print(model)

In [ ]:
model.to(device)

In [28]:
transform = torchvision.transforms.Compose([torchvision.transforms.Resize((128, 128)),
                                            torchvision.transforms.ToTensor()])

In [29]:
class Celeba(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, validation=False):
        self.img_labels = pd.read_csv(annotations_file)
        valid_images = os.listdir(img_dir)
        if validation == True:
            valid_images = valid_images[int(0.8 * len(valid_images)):]
        else:
            valid_images = valid_images[:int(0.8 * len(valid_images))]    
        self.img_labels = self.img_labels[self.img_labels["image_id"].isin(valid_images)]
        self.img_labels = self.img_labels.reset_index()
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.loc[idx, "image_id"])
        image = Image.open(img_path)
        label = self.img_labels.loc[idx, "Smiling"]
        if label == -1:
            label = 0
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [30]:
ann_file = "/content/drive/MyDrive/DataScience/datasets/Celeba/img_align_celeba/list_attr_celeba.csv"
img_dir = "/content/drive/MyDrive/DataScience/datasets/Celeba/img_align_celeba/img_align_celeba"

In [31]:
Celeba_dataset_train = Celeba(annotations_file=ann_file, img_dir=img_dir, transform=transform, validation=False)
Celeba_dataset_validation = Celeba(annotations_file=ann_file, img_dir=img_dir, transform=transform, validation=True)

In [ ]:
# print(Celeba_dataset_train.__len__())
print(Celeba_dataset_train.__getitem__(11000))

In [74]:
trainloader = DataLoader(Celeba_dataset_train, batch_size=64, shuffle=True, num_workers=2)
validationloader = DataLoader(Celeba_dataset_validation, batch_size=64, shuffle=False, num_workers=2)

In [24]:
trainloader

In [34]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
from time import time
import os
from tqdm import tqdm 
num_epochs = 20

checkpoint_dir = "/content/drive/MyDrive/DataScience/datasets/Celeba/checkpoint_experiment1"
if os.path.isdir(checkpoint_dir) == False:
    os.makedirs(checkpoint_dir)

steps_to_checkpoint = 50

for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    model.train()
    for i, data in tqdm(enumerate(trainloader, 0)):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)        
        loss.backward()        
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.3f}')
            running_loss = 0.0
        if i % steps_to_checkpoint == 49:
            torch.save(model.state_dict(), os.path.join(checkpoint_dir, "model_step_" + str(i) + ".pth"))


    model.eval() # opresc orice actualizare de parametri        
    running_loss = 0
    corrects = 0
    for j, data in tqdm(enumerate(validationloader, 0)):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        with torch.no_grad(): 
          outputs = model(inputs)
          prob_outputs = nn.Softmax(dim=-1)(outputs)
        predicted_labels = torch.argmax(prob_outputs, dim=1)   
        loss = criterion(outputs, labels)          
        running_loss += loss.item()
        corrects += torch.sum(predicted_labels == labels)
    print(f'[{epoch + 1}] loss: {running_loss / (j+1):.3f}')
    print("Acuracy is ", corrects.item() / ((j+1)*64))
                  

print('Finished Training')

4it [00:54, 11.09s/it]